In [1]:
import io
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

device = "cpu"

In [2]:

df = pd.read_csv('/Users/aleksandragorbuncova/PycharmProjects/lab-py-3sem/lab3/datasetAll/dataset.csv')

df.columns = ['Datatime', 'temp_day', 'wind', 'pressure_day']
df = df.drop(columns = ['wind', 'pressure_day'])

nan_value = float("NaN")                             # 3
df.replace(" ", nan_value, inplace=True)
df = df.dropna()

df[['temp_day']] = df[['temp_day']].astype(int)  # 4
df_save = df
print(df)

        Datatime  temp_day
0     2008-01-11       -20
1     2008-01-12       -20
2     2008-01-13        -9
3     2008-01-14        -6
4     2008-01-15        -1
...          ...       ...
5438  2022-12-13        -6
5439  2022-12-14        -5
5440  2022-12-15        -5
5441  2022-12-16        -4
5442  2022-12-17        -3

[5434 rows x 2 columns]
